# Feature Selection
<br>
Referred to https://www.sciencedirect.com/science/article/pii/S1877050922007955 by Fátima Rodriguesa
and Ângelo Pintob

- Boruta algorithm was used. Boruta is a heuristic variable selection algorithm based on the random forests algorithm that aims to find the most relevant variables in a dataset.
- The Boruta algorithm show the relevant and non-relevant variables. This algorithm was used because it does not look for a suboptimal solution, instead it tries to find all variables with relevant information, thus allowing to eliminate variables that would negatively affect the forecast models

In [1]:
!pip install boruta

In [2]:
import pandas as pd
import numpy as np
import pickle
import json
from sklearn.impute import SimpleImputer
from constants.constants import *
from models import Boruta, PreProcessor # see models.py for implementation

pd.set_option('display.max_columns', None) # show all columns
pd.set_option('display.max_rows', 20) # show all rows
pd.set_option('display.max_seq_items', None)

To keep it simple for now, we will just use the outcome to filter out the most important features. We should only use the data from 2019 to 2022 as we are predicting 2023

In [3]:
dfs = []

for year in range(2019, 2023):
    df = pd.read_csv(f'./data/machine_learning/outcome/{str(year)}.csv')
    dfs.append(df)

train_df = pd.concat(dfs, ignore_index=True)

In [4]:
train_df = train_df.drop(['form', 'opponent_form'], axis=1) # too complicated to take into account form when predicting

,season_start_year,year,weekday,month,day,hour,minute,team,opponent,is_home,form,opponent_form,team_minutes_90s,team_country,team_squad_size,team_foreigners,team_squad_value_avg,team_squad_value,team_players_used,team_gd_per90,team_pens_made_per90,team_pens_att_per90,team_cards_yellow_per90,team_cards_red_per90,team_cards_yellow_red_per90,team_fouls_per90,team_fouled_per90,team_offsides_per90,team_own_goals_per90,team_gk_goals_against_per90,team_gk_save_pct,team_gk_clean_sheets_pct,team_gk_psxg,team_gk_psnpxg_per_shot_on_target_against,team_gk_psxg_net_per90,team_gk_passes_pct_launched,team_gk_pct_passes_launched,team_gk_goal_kick_length_avg,team_gk_crosses_stopped_pct,team_gk_def_actions_outside_pen_area_per90,team_gk_shots_on_target_against_per90,team_gk_pens_saved_per90,team_gk_pens_missed_per90,team_gk_free_kick_goals_against_per90,team_gk_corner_kick_goals_against_per90,team_goals_per90,team_assists_per90,team_goals_assists_per90,team_goals_pens_per90,team_goals_assists_pens_per90,team_xg_per90,team_npxg_per90,team_xg_assist_per90,team_xg_xg_assist_per90,team_npxg_xg_assist_per90,team_shots_on_target_pct,team_shots_per90,team_shots_on_target_per90,team_goals_per_shot,team_goals_per_shot_on_target,team_average_shot_distance,team_npxg_per_shot,team_passes_pct,team_passes_pct_short,team_passes_pct_medium,team_passes_pct_long,team_passes_total_distance_per90,team_progressive_passes_per90,team_passes_progressive_distance_per90,team_pass_xa_per90,team_xg_assist_net_per90,team_assisted_shots_per90,team_passes_into_final_third_per90,team_passes_into_penalty_area_per90,team_crosses_into_penalty_area_per90,team_passes_live_per90,team_passes_dead_per90,team_passes_free_kicks_per90,team_through_balls_per90,team_passes_switches_per90,team_crosses_per90,team_throw_ins_per90,team_corner_kicks_per90,team_corner_kicks_in_per90,team_corner_kicks_out_per90,team_corner_kicks_straight_per90,team_sca_per90,team_gca_per90,team_sca_passes_live_per90,team_sca_passes_dead_per90,team_sca_take_ons_per90,team_sca_shots_per90,team_sca_fouled_per90,team_sca_defense_per90,team_gca_passes_live_per90,team_gca_passes_dead_per90,team_gca_take_ons_per90,team_gca_shots_per90,team_gca_fouled_per90,team_gca_defense_per90,team_take_ons_won_pct,team_take_ons_tackled_pct,team_possession,team_touches_per90,team_touches_def_pen_area_per90,team_touches_def_3rd_per90,team_touches_mid_3rd_per90,team_touches_att_3rd_per90,team_touches_att_pen_area_per90,team_touches_live_ball_per90,team_take_ons_per90,team_carries_per90,team_carries_distance_per90,team_progressive_carries_per90,team_carries_progressive_distance_per90,team_carries_into_final_third_per90,team_carries_into_penalty_area_per90,team_miscontrols_per90,team_dispossessed_per90,team_challenge_tackles_pct,team_aerials_won_pct,team_tackles_per90,team_tackles_won_per90,team_tackles_def_3rd_per90,team_tackles_mid_3rd_per90,team_tackles_att_3rd_per90,team_challenge_tackles_per90,team_challenges_per90,team_challenges_lost_per90,team_blocks_per90,team_blocked_shots_per90,team_blocked_passes_per90,team_interceptions_per90,team_tackles_interceptions_per90,team_clearances_per90,team_errors_per90,team_ball_recoveries_per90,team_aerials_won_per90,team_aerials_lost_per90,opponent_minutes_90s,opponent_country,opponent_squad_size,opponent_foreigners,opponent_squad_value_avg,opponent_squad_value,opponent_players_used,opponent_gd_per90,opponent_pens_made_per90,opponent_pens_att_per90,opponent_cards_yellow_per90,opponent_cards_red_per90,opponent_cards_yellow_red_per90,opponent_fouls_per90,opponent_fouled_per90,opponent_offsides_per90,opponent_own_goals_per90,opponent_gk_goals_against_per90,opponent_gk_save_pct,opponent_gk_clean_sheets_pct,opponent_gk_psxg,opponent_gk_psnpxg_per_shot_on_target_against,opponent_gk_psxg_net_per90,opponent_gk_passes_pct_launched,opponent_gk_pct_passes_launched,opponent_gk_goal_kick_length_avg,opponent_gk_crosses_stopped_pct,opponent_gk_def_actions_outside_pen_area_per90,opponent_gk_shots_on_target_aga

We create a Preprocessor class and Boruta child class in models.py so that it can have access to the preprocessor

Normalize the data

In [5]:
train_df_clean = train_df.dropna()
X = train_df_clean.drop(columns=['outcome'])
y = train_df_clean['outcome']
preprocessor = PreProcessor(X, y)

# Logistic Regression

Trying to use Logistic Regression to further reduce the number of features

In [6]:
with open('data/machine_learning/pkl/processor.pkl', 'rb') as f:
    preprocessor = pickle.load(f)

In [7]:
from sklearn.linear_model import LogisticRegression

# Initialize and fit Logistic Regression with L1 penalty
log_reg = LogisticRegression(penalty='l1', solver='saga', C=1.0, max_iter=10000, multi_class='multinomial', random_state=42)
# Fit your logistic regression model
log_reg.fit(preprocessor.df_train_preprocessed, preprocessor.y_train_encoded)

LogisticRegression(max_iter=10000, multi_class='multinomial', penalty='l1',
                   random_state=42, solver='saga')

In [8]:
feature_names = preprocessor.df_train_preprocessed.columns
coefficients = log_reg.coef_
class_labels = preprocessor.target_encoder.inverse_transform(log_reg.classes_)

coef_df = pd.DataFrame(coefficients.T,  # Transpose to make features as rows
                       columns=[f'Coeff_Class_{cls}' for cls in class_labels],
                       index=feature_names)

coef_df

,Coeff_Class_D,Coeff_Class_L,Coeff_Class_W
day,0.021551,0.000000,-0.025881
form,0.018492,-0.061269,0.000000
hour,0.004605,-0.007555,0.000000
is_home,0.000000,-0.168583,0.194414
minute,0.016961,-0.022936,0.000000
...,...,...,...
team_country_ENG,0.001083,0.000000,-0.073859
team_country_FRA,-0.031006,0.044002,0.000000
team_country_GER,-0.150584,0.000000,0.000000
team_country_ITA,0.000000,-0.050712,0.001613


In [9]:
# Identify features with zero coefficients
threshold = 0.1
weak_features_mask = (coef_df['Coeff_Class_D'] < threshold) & (coef_df['Coeff_Class_L'] < threshold) & (coef_df['Coeff_Class_W'] < threshold)
strong_features_mask = ~weak_features_mask
coef_df[weak_features_mask]

,Coeff_Class_D,Coeff_Class_L,Coeff_Class_W
day,0.021551,0.000000,-0.025881
form,0.018492,-0.061269,0.000000
hour,0.004605,-0.007555,0.000000
minute,0.016961,-0.022936,0.000000
month,-0.013872,0.001974,0.000000
...,...,...,...
team_country_ENG,0.001083,0.000000,-0.073859
team_country_FRA,-0.031006,0.044002,0.000000
team_country_GER,-0.150584,0.000000,0.000000
team_country_ITA,0.000000,-0.050712,0.001613


In [10]:
strong_features_df = coef_df[strong_features_mask]
strong_features_df

,Coeff_Class_D,Coeff_Class_L,Coeff_Class_W
is_home,0.000000,-0.168583,0.194414
opponent_aerials_won_pct,0.118672,0.000000,-0.039114
opponent_aerials_won_per90,-0.044429,0.000000,0.173660
opponent_assisted_shots_per90,0.167085,0.000000,-0.252517
opponent_carries_per90,0.000000,0.000000,0.174517
...,...,...,...
team_shots_per90,0.000000,0.000000,0.701233
team_squad_value_avg,0.000000,-0.243025,0.157051
team_touches_att_3rd_per90,0.000000,0.113958,0.000000
team_xg_assist_per90,0.000000,0.246370,-0.511695


In [11]:
LOG_REG_COLS = strong_features_df.index.tolist()

In [12]:
boruta = Boruta()

boruta.fit(preprocessor.df_train_preprocessed[LOG_REG_COLS], preprocessor.y_train)
BORUTA_FEATURES = boruta.get_selected_features()

with open(BORUTA_FEATURES_PATH, 'w') as file:
    json.dump(BORUTA_FEATURES, file, indent=4)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	54
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	54
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	54
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	54
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	54
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	54
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	54
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	38
Tentative: 	6
Rejected: 	10
Iteration: 	9 / 100
Confirmed: 	38
Tentative: 	6
Rejected: 	10
Iteration: 	10 / 100
Confirmed: 	38
Tentative: 	6
Rejected: 	10
Iteration: 	11 / 100
Confirmed: 	38
Tentative: 	6
Rejected: 	10
Iteration: 	12 / 100
Confirmed: 	38
Tentative: 	3
Rejected: 	13
Iteration: 	13 / 100
Confirmed: 	38
Tentative: 	3
Rejected: 	13
Iteration: 	14 / 100
Confirmed: 	38
Tentative: 	3
Rejected: 	13
Iteration: 	15 / 100
Confirmed: 	38
Tentative: 	3
Rejected: 	13
Iteration: 	16 / 100
Confirmed: 	38
Tentative: 	3
Reject

In [13]:
print(BORUTA_FEATURES)

['is_home', 'opponent_assisted_shots_per90', 'opponent_carries_per90', 'opponent_gd_per90', 'opponent_gk_pct_passes_launched', 'opponent_gk_psxg', 'opponent_goals_pens_per90', 'opponent_goals_per90', 'opponent_passes_dead_per90', 'opponent_passes_live_per90', 'opponent_passes_pct', 'opponent_passes_pct_short', 'opponent_progressive_passes_per90', 'opponent_shots_on_target_per90', 'opponent_shots_per90', 'opponent_squad_value_avg', 'opponent_touches_att_3rd_per90', 'opponent_touches_mid_3rd_per90', 'opponent_xg_assist_per90', 'opponent_xg_per90', 'team_assisted_shots_per90', 'team_carries_distance_per90', 'team_carries_per90', 'team_carries_progressive_distance_per90', 'team_gca_per90', 'team_gd_per90', 'team_goals_per90', 'team_passes_dead_per90', 'team_passes_live_per90', 'team_passes_pct', 'team_passes_pct_short', 'team_passes_total_distance_per90', 'team_progressive_passes_per90', 'team_shots_on_target_per90', 'team_shots_per90', 'team_squad_value_avg', 'team_touches_att_3rd_per90',